In [41]:
import os
from vlc import *
import hashlib

class EventHook(object):
	def __init__(self):
		self.__handlers = []

	def __iadd__(self, handler):
		self.__handlers.append(handler)
		return self

	def __isub__(self, handler):
		self.__handlers.remove(handler)
		return self

	def fire(self, *args, **keywargs):
		for handler in self.__handlers:
			handler(*args, **keywargs)

class MediaLibrary():
    def __init__(self, import_path = "."):
        instance = Instance()
        self.lib = []
        for root, dirs, files in os.walk(import_path, topdown=False):
            for name in files:
                if name.endswith(".mp3"):
                    self.lib.append(AudioMedia(instance, os.path.join(root, name)))
    
    def search(self, needle):
        results = []
        for media in self.lib:
            matching = [(key, val) for key, val in media.meta.items() 
                        if (val is not None and needle is not None) and (needle.lower() in str(val).lower())]
            if len(matching) > 0:
                results.append(media)
        return results


class AudioMedia():
    def get_hash(self):
        return hashlib.sha256(self.media_loc.encode()).hexdigest()
    
    def __init__(self, vlc_instance, media):
        if (os.path.isfile(media)):
            self.instance = vlc_instance
            self.media_loc = media
            self.media = self.instance.media_new(media)
            self.media.parse()
            self.meta = self._get_media_meta()
            self.art_path = None
            self.on_art_ready = EventHook()
        else: 
            raise IOError('File ' + media + ' not found')

    def fetch_media_art(self):
        if self.art_path is not None:
            return self.art_path

        if (self.meta["Artist"] is None) or (self.meta["Album"] is None):
            return None

        artdir = "artlib"

        path_concat = os.path.join("static", artdir)
        if not os.path.isdir(path_concat):
            os.makedirs(path_concat)

        filename = self.meta["Artist"] + "-" + self.meta["Album"]
        filename = unicode(unicodedata.normalize('NFKD', filename).encode('ascii', 'ignore'))
        filename = unicode(re.sub('[^\w\s-]', '', filename).strip().lower())
        filename = unicode(re.sub('[-\s]+', '-', filename)) + ".jpg"    
        path_concat = os.path.join(path_concat, filename)
        
        if os.path.isfile(path_concat):
            self.art_path = os.path.join(artdir, filename)
            return self.art_path

        if sacad.search_and_download(
                self.meta["Artist"], 
                self.meta["Album"], 
                sacad.cover.CoverImageFormat.JPEG,
                600,
                25,
                [],
                False,
                path_concat):
            self.art_path = os.path.join(artdir, filename)
            return self.art_path
        else:
            return None

    def _get_media_meta(self):
        meta = {}
        for meta_code, meta_name in Meta._enum_names_.items():
            if (meta_code != 23) and (meta_code != 24) and (meta_code != 25): #these meta codes don't work for some reason
                meta[meta_name] = self.media.get_meta(meta_code)
        meta["length"] = self.media.get_duration()
        return meta



In [22]:
import hashlib
class AudioMedia():
    def __init__(self):
        self.path_hash = int(hashlib.sha256("lol abcd".encode()).hexdigest(), 16)

    def __hash__(self):
        print("here you go")
        print(self.path_hash)
        return self.path_hash

    def _get_media_meta(self):
        print(hash(self))

a = AudioMedia()
a._get_media_meta()
print(hash(a))

here you go
113868031188330780213630867705826553957428895603776635249782344416048461488873
40128193470625520
here you go
113868031188330780213630867705826553957428895603776635249782344416048461488873
40128193470625520


----------
here you go
113868031188330780213630867705826553957428895603776635249782344416048461488873
123
----------


In [24]:
1 >= 1

True